# Exploratory Data Analysis

Notebook dedicado para fazer análises dos valores, e possíveis inferências que podemos fazer em cima deles.
Fazendo análises de atributos de forma individual, e também correlações.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', None)
sns.set_theme()

In [ ]:
df = pd.read_csv('../data/processed/dados_ficha_a_desafio_processed.csv', 
                 parse_dates=[
                     'data_nascimento', 
                     'data_atualizacao_cadastro',
                     'updated_at',
                     'data_cadastro'
])

In [ ]:
# df.info()

## Objetivo e indagações

Após fazer um pré processamento considerável, é possível pensar que em qual é o objetivo por trás desse dataset, e o que podemos extrair dele, através das análises dos registros, parece estar relacionado com uma ficha médica que é preenchida com as principais informações sobre o paciente, claro que excluindo dados pessoais, sendo possível deixar anonimizado, e mantém os principais dados das doenças, meios de transporte, comunicações e características físicas e outros detalhes. 

Esses dados podem estar relacionado alguma rede de saúde, que retém essas informações para usos futuros, seja para análises, predições ou até levatar dados passados sobre o usuário. Tendo isso em mente, como podemos levantar alguns insight?
* Análise Univariada (focando somente na própria variável);
* Análise Multivariada (principalmente com o foco em pares);

# Análise Univariada

O objetivo aqui não é ficar tão repetitivo, mas tentar trazer os principais pontos que chamaram a atenção para tentar trazer algum insight que agregue no fim.

### Atributo: Id_paciente

In [ ]:
# df['id_paciente'].value_counts()

In [ ]:
# df[df['id_paciente']=='933e58c0-1175-46d8-a1e6-4a534a257feb']

Existe alguns valores duplicados em registros no atributo de 'id_paciente' que dá entender inicialmente que seriam para um mesmo paciente, porém não são de fato, quando vamos ver as características de cada registro, não são diferenças pequenas, são consideráveis, o que dá para entender que retratam indivíduos, diferentes. Provavelmente pode ter vindo de sistemas diferentes, mas coincidentalmente registraram o mesmo id para esses pacientes.

O ideal para mim na situação seria tratar como identificações diferentes, criando algum identificador novo, podendo deixar essa coluna de 'id_paciente' de lado a princípio.


### Atributo: Renda Familiar

Começando com uma análise dos seus valores e suas quantidades:

In [ ]:
# df['renda_familiar'].value_counts()

Com esses dados é possível ver como é a distribuição de renda dentre os indivíduos, onde podemos checar que mais de 75% dos indivíduos tem como renda_familiar até 2 Salários Mínimos. A princípio não sei dizer que representa bem a distribuição de renda brasileira, porém é possível fazer pesquisas com os dados do IBGE, porém pode ser do escopo da análise.

In [ ]:
order_1 = ['1/4 Salário Mínimo',
 '1/2 Salário Mínimo',
 '1 Salário Mínimo',
 '2 Salários Mínimos',
 '3 Salários Mínimos',
 '4 Salários Mínimos',
 'Mais de 4 Salários Mínimos']


ax = sns.barplot(
    df['renda_familiar'].value_counts(), order=order_1
)
plt.xticks(rotation=0, labels=['1/4', '1/2','1', '2', '3', '4', '4+'], ticks=order_1)
plt.title('Renda Familar (por quantidade de Salários Mínimos)')
plt.xlabel("Quantidade de Salários Mínimos")
plt.ylabel("Quantidade de registros")
plt.show()

### Atributo: Escolaridade

De forma parecida podemos ver como os pacientes são retratados aqui através da escolaridade, fazendo uma visualiação muito parecida.

In [ ]:
order_2 = [
    'Não sabe ler/escrever',
    'Fundamental Incompleto',
    'Fundamental Completo',
    'Alfabetizado',
    'Médio Incompleto',
    'Médio Completo',
    'Superior incompleto',
    'Superior completo',
    'Especialização/Residência',
    'Mestrado',
    'Doutorado']


ax = sns.barplot(
    df['escolaridade'].value_counts(), order=order_2
)
plt.xticks(
    rotation=-40, 
    labels=[
        'Não sabe ler', 'Fund. Inc', 'Fund. Comp',
        'Alfab', 'Med. Inc.', 'Med. Comp', 'Sup. Inc.',
        'Sup. Comp.', 'Esp/Res.', 'Mest', 'Doc'
    ],
    ticks=order_2
)
plt.title('Escolaridade dos Registros')
plt.xlabel("Grau de Escolaridade")
plt.ylabel("Quantidade de registros")
plt.show()

Podemos ver que grande parte dos dados dos pacientes mostram que maior parte deles tem Fundamental Incompleto e Ensino Médio Completo, e de forma ensino superior em diante são aqueles que estão após, são os que menos tem registros. Novamente poderíamos tentar fazer comparações com dados do IBGE para ver se é uma retratação, porém sai do escopo. Acredito que é mais interessante ver esses valores agora sendo cruzados com os de outros atributos.

### Atributo: Criança matriculada em creche, pré escola (Avançando um pouco para o multivalorado)

Através desse atributo, e segundo a descrição de 'Indicação se a criança está matriculada em creche ou pré-escola', podemos ver quais são os usuários tem filhos matriculados em creches e pré-escolas, ou se a criança própria estaria matriculada. Indo um pouco mais a fundo para tentar entender qual o melhor conceito que se encaixa, adicionei a atributo de 'data_nascimento' para checar o comportamento dos registros que contém crianças na creche.

In [ ]:
df_contem_crianca = df[['data_nascimento', 'crianca_matriculada_creche_pre_escola']][df['crianca_matriculada_creche_pre_escola']==1].sort_values(by='data_nascimento')

In [ ]:
df_contem_crianca['ano'] = df_contem_crianca['data_nascimento'].dt.year
df_agrupado = df_contem_crianca.groupby('ano').size().reset_index(name='quantidade')

# Plotando o scatterplot
plt.figure(figsize=(10, 5))
ax = sns.scatterplot(data=df_agrupado, x='ano', y='quantidade')
ax.set_xlabel('Ano de Nascimento')
ax.set_ylabel('Quantidade de Crianças Matriculadas')
plt.xticks(rotation=45)
plt.show()

Com esse gráfico podemos pensar que grande parte dos registros contam como se seus filhos tivessem em época de creche, porém também há dados que mostram como se o próprio paciente fosse a creche ou pré-escola e também a o outro extremo, de paciente que nasceram em torno de 1920, que contém criança na creche ou pré-escola, como se seu neto ou bisneto satisfazesse essa condição.

Claro que não podemos deixar de pensar que tenha dados inválidos ou incorretos, mas não fica claro essa correlação entre os atributos, não tendo um significado exato.

### Atributo: Meio de transporte


In [ ]:
colunas_transporte = [
    'meios_transporte_alternativo', 'meios_transporte_animal', 'meios_transporte_bicicleta',
    'meios_transporte_caminhao', 'meios_transporte_carro', 'meios_transporte_carroca',
    'meios_transporte_maritimo', 'meios_transporte_metro', 'meios_transporte_outros',
    'meios_transporte_trem', 'meios_transporte_onibus'
]

# Contar quantos valores "True" existem em cada coluna
contagem_transporte = df[colunas_transporte].sum().sort_values(ascending=False)

# Criar rótulos mais curtos removendo "meios_transporte_"
nomes_formatados = [nome.replace("meios_transporte_", "").capitalize() for nome in contagem_transporte.index]

# Criar um gráfico de barras
plt.figure(figsize=(12, 6))
ax = sns.barplot(x=contagem_transporte.values, y=nomes_formatados)

# Ajustar os rótulos
plt.xlabel("Quantidade de Pessoas que Utilizam")
plt.ylabel("Meios de Transporte")
plt.title("Uso de Diferentes Meios de Transporte")
plt.show()

Lembrando que para esse atributo, era um caso multivalorado e poderia ter valores nulos, a soma total dessa contagem não deve estar relacionada com a quantidade de registros.

Maior parte das pessoas utilizam ônibus e carro, isso alguma forma pode se relacionar com a renda familiar que vimos anteriormente, mas provavelmente devemos levar mais fatores em consideração.

Poderíamos repetir isso para as outras colunas que eram multivaloradas, porém isso pode ser mais repetitivo, com essas análises conseguimos compreender um pouco melhor o dataset que estamos trabalhando e podemos tentar correr atrás agora de algumas análises multivaloradas.

---

# Análise Multivalorada

Como maior parte dos dados é multivariado, a principal forma para mim de comparar dados será fazer pivot_tables que eu possa ter uma visão mais adequada sobre os dados, talvez até um heatmap.

### Renda Familiar X Situação Profissional

Vamos ver se os dados estão consistentes e caso haja alguma discrepância analisar o motivo.

In [ ]:
pivot = pd.crosstab(df['situacao_profissional'], df['renda_familiar'], 
                                   values=df['renda_familiar'], aggfunc='size', 
                                   dropna=False, normalize=True) * 100
pivot2 = pivot[order_1].copy()
# pivot2

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(pivot2, annot=True, cmap='YlGnBu', fmt='.2f', cbar=True)
plt.title("Heatmap de Percentual - Renda Familiar vs Situação Profissional")
plt.show()

Esses dados mostram como é a relação entre os atributos, os números dentro da tabela representam a contagem de indivíduos em cada combinação de situação profissional e faixa de renda, e como esperado maior partes os registros ficam em volta de emprego formal, e as rendas de 1 e 2 Salários mínimos. Isso indica que o emprego formal está fortemente associado a rendas familiares mais elevadas. Ao mesmo tempo, a desigualdade de renda é evidente, com pessoas desempregadas e em trabalhos informais concentradas nas faixas de renda mais baixas.

Temos alguns casos que são Não definidos, porém podem ser entendidos com uma análise mais profunda nesses registros.

---

### Meios de Comunicação X Escolaridade.

Essa análise, pensando no contexto dos dados originais, porém ser interessantes para encontrar meios mais práticos de entrar em contato com a pessoa através do nível de escolaridade, ou quem sabe se for mais adequado data de nascimento, pode ser feito uma continuação da análise para tentar ir mais longe em como é o melhor meio de entrar em contato com o paciente.

In [ ]:
meios_comunicacao = [
    'meios_comunicacao_grupos_religiosos', 'meios_comunicacao_internet', 
    'meios_comunicacao_jornal', 'meios_comunicacao_outros',
    'meios_comunicacao_revista', 'meios_comunicacao_radio', 'meios_comunicacao_televisao'
]

crosstabs = []

for meio in meios_comunicacao:
    # Criar a crosstab filtrando apenas os valores True
    crosstab_temp = pd.crosstab(df['escolaridade'], df[meio])

    # Manter apenas a coluna True e renomeá-la com o nome do meio de comunicação
    if True in crosstab_temp.columns:
        crosstab_temp = crosstab_temp[True]
        crosstab_temp.name = meio.replace("meios_comunicacao_", "")  # Removendo prefixo
    else:
        # Se não houver valores True, criar uma coluna zerada
        crosstab_temp = pd.Series(0, index=df['escolaridade'].unique(), name=meio.replace("meios_comunicacao_", ""))

    crosstabs.append(crosstab_temp)

# Concatenar todos os crosstabs em uma única tabela
crosstab_final = pd.concat(crosstabs, axis=1)


In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(crosstab_final, annot=True, cmap='YlGnBu', fmt='.2f', cbar=True)
plt.title("Heatmap de Meios de Comunicação vs Escolaridade")
plt.show()

Com esse heatmap, podemos visualizar quais os principais meios de comunicação por escolaridade, de cara vemos que a escolaridade de médio completo e o fundamental incompleto são os que mais tem registros, porém eles também são os que mais tem pelo dataset. Interessante também que no meio de comunicação, não temos celulares ou se eles já englobam em internet. Visto que são dados atuais, cabe o questionamento sobre esses meios de comunicações.

Além disso, podemos ver pelas linhas que o meio de comunicação pelo radio ainda é bastante utilizado, e cada vez menos jornais e revistas, meios físicos estão saindo cada vez mais para meios digitais. Isso para qualquer nível de escolaridade.

Poderíamos tentar acrescentar mais detalhes sobre esses dados introduzindo mais questões sobre data de nascimento e até de renda familiar, porém traria complexidades muito maiores.

---

De forma geral, poderia ter feito mais análise, porém para não ficar tão extenso escolhi fazer dos principais atributos que me chamaram a atenção. Porém tem uma vasta combinações de atributos e ideias para explorar e entender mais como os dados estão distribuídos e propriamente entender o contexto desses dados.